## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 2 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

# Gesture Classification using TAO GestureNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080"> 

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train a GestureNet model on HGR dataset
* Run Inference on the trained model
* Export the retrained model to a .etlt file for deployment to DeepStream SDK

### Table of Contents

This notebook shows an example of classifying gestures using GestureNet in the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
    A. [Verify and prepare dataset](#head-1-1) <br>
    B. [Generate hand crops and dataset json](#head-1-2) <br>
    C. [Download pre-trained model](#head-1-3) <br>
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
    2.3 [Setup Python environment](#head-2-3) <br>
    2.4 [Reset env variables](#head-2-4) <br>
3. [Dataset convert](#head-3) <br>
4. [Provide training specification](#head-4) <br>
5. [Run TAO training](#head-5) <br>
6. [Evaluate the trained model](#head-6) <br>
7. [Export](#head-7) <br>
8. [Inference](#head-8) <br>

#### Note
1. This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly
2. This notebook uses HGR dataset by default, which should be around ~128MB.
3. Using the default config/spec file provided in this notebook, each weight file size of gesturenet created during training will be ~45 MB

## 0. Set up env variables and set FIXME parameters <a class="anchor" id="head-0"></a>

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

#### FIXME
1. NUM_GPUS - set this to <= number of GPU's availble on the instance
1. EXPERIMENT_DIR - set this path to a folder location where pretrained models, checkpoints and log files during different model actions will be saved
1. delete_existing_experiments - set to True to remove existing pretrained models, checkpoints and log files of a previous experiment
1. DATA_DIR - set this path to a folder location where you want to dataset to be present
1. delete_existing_data - set this to True to remove existing preprocessed and original data

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
#FIXME1
%env NUM_GPUS=1

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/ColabNotebooks
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise("Error, enter the path of the colab notebooks repo correctly")

#FIXME2
%env EXPERIMENT_DIR=/content/drive/MyDrive/results/gesturenet
#FIXME3
delete_existing_experiments = True
#FIXME4
%env DATA_DIR=/content/drive/MyDrive/gesturenet_data/
#FIXME5
delete_existing_data = False

if delete_existing_experiments:
    !sudo rm -rf $EXPERIMENT_DIR
if delete_existing_data:
    !sudo rm -rf $DATA_DIR

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/gesturenet/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

!sudo mkdir -p $DATA_DIR && sudo chmod -R 777 $DATA_DIR
!sudo mkdir -p $EXPERIMENT_DIR && sudo chmod -R 777 $EXPERIMENT_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

We will be using the database for hand gesture recognition (HGR) for the tutorial. To find more details, please visit http://sun.aei.polsl.pl/~mkawulok/gestures/. Please download the HGR1 [images](http://sun.aei.polsl.pl/~mkawulok/gestures/hgr1_images.zip), [feature points](http://sun.aei.polsl.pl/~mkawulok/gestures/hgr1_feature_pts.zip) and HGR2B [images](http://sun.aei.polsl.pl/~mkawulok/gestures/hgr2b_images.zip), [feature points](http://sun.aei.polsl.pl/~mkawulok/gestures/hgr2b_feature_pts.zip) and place the zip files in `$DATA_DIR`. 

### A. Verify and prepare dataset <a class="anchor" id="head-1-1"></a>

In [ ]:
# Check the zip files are present.
!if [ ! -f $DATA_DIR/hgr1_images.zip ]; then echo 'hgr1_images zip file not found, please download.'; else echo 'Found hgr1_images zip file.';fi
!if [ ! -f $DATA_DIR/hgr1_feature_pts.zip ]; then echo 'hgr1_feature_pts zip file not found, please download.'; else echo 'Found hgr1_feature_pts zip file.';fi
!if [ ! -f $DATA_DIR/hgr2b_images.zip ]; then echo 'hgr2b_images zip file not found, please download.'; else echo 'Found hgr2b_images zip file.';fi
!if [ ! -f $DATA_DIR/hgr2b_feature_pts.zip ]; then echo 'hgr2b_feature_pts zip file not found, please download.'; else echo 'Found hgr2b_feature_pts zip file.';fi

In [ ]:
# unpack downloaded datasets to $DATA_DIR.
# The images will be under $DATA_DIR/original_images and $DATA_DIR/feature_points
!unzip -u ${DATA_DIR}/hgr1_images.zip -d ${DATA_DIR}
!unzip -u ${DATA_DIR}/hgr1_feature_pts.zip -d ${DATA_DIR}
!unzip -u ${DATA_DIR}/hgr2b_images.zip -d ${DATA_DIR}
!unzip -u ${DATA_DIR}/hgr2b_feature_pts.zip -d ${DATA_DIR}

In [ ]:
# Convert dataset to required format for gesturenet dataset_convert
!python3 $COLAB_NOTEBOOKS_PATH/tensorflow/gesturenet/convert_hgr_to_tlt_data.py --input_image_dir=$DATA_DIR/original_images \
                                      --input_label_file=$DATA_DIR/feature_points \
                                      --output_dir=$DATA_DIR

In [ ]:
# verify
import os

DATA_DIR = os.environ.get('DATA_DIR')
num_labels = len(os.listdir(os.path.join(DATA_DIR, "original/data/annotation")))
print("Number of labels in the dataset. {}".format(num_labels))

### C. Download pre-trained model <a class="anchor" id="head-1-3"></a>

Please follow the instructions in the following to download and verify the pretrained model for gesturenet.

For FpeNet pretrained model please download model: `nvidia/tao/gesturenet:trainable_v1.0`.

After obtaining the pre-trained model, please place the model in $EXPERIMENT_DIR

You will have the following path-

* pretrained model in `$EXPERIMENT_DIR/pretrained_models/gesturenet_vtrainable_v1.0/model.tlt`

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/ngc_content/
%env CLI=ngccli_cat_linux.zip
!sudo mkdir -p $LOCAL_PROJECT_DIR/ngccli && sudo chmod -R 777 $LOCAL_PROJECT_DIR

# Remove any previously existing CLI installations
!sudo rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/gesturenet:*

In [ ]:
# Create the target destination to download the model.
!mkdir -p $EXPERIMENT_DIR/pretrained_models/

In [ ]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/gesturenet:trainable_v1.0 \
    --dest $EXPERIMENT_DIR/pretrained_models/

In [ ]:
!ls -rlt $EXPERIMENT_DIR/pretrained_models/gesturenet_vtrainable_v1.0 

In [ ]:
# Check the model is present
!if [ ! -f $EXPERIMENT_DIR/pretrained_models/gesturenet_vtrainable_v1.0/model.tlt ]; then echo 'Pretrained model file not found, please download.'; else echo 'Found Pretrain model file.';fi

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
#FIXME
%env GENERAL_WHL_PATH=/content/drive/MyDrive/tf/general_whl
#FIXME
%env CODEBASE_WHL_PATH=/content/drive/MyDrive/tf/codebase_whl

import os
if os.path.exists(os.environ["GENERAL_WHL_PATH"]) and os.path.exists(os.environ["GENERAL_WHL_PATH"]):
    if os.environ["GOOGLE_COLAB"] == "1":
        os.environ["bash_script"] = "setup_env.sh"
    else:
        os.environ["bash_script"] = "setup_env_desktop.sh"

    !sed -i "s|PATH_TO_GENERAL_WHL|$GENERAL_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
    !sed -i "s|PATH_TO_CODEBASE_WHL|$CODEBASE_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
    !sed -i "s|PATH_TO_COLAB_NOTEBOOKS|$COLAB_NOTEBOOKS_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

    !sh $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
else:
    raise("Error, enter the whl paths correctly")

In [ ]:
if os.environ.get("PYTHONPATH","") == "":
    os.environ["PYTHONPATH"] = ""
os.environ["PYTHONPATH"]+=":/opt/nvidia/"
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["PYTHONPATH"]+=":/usr/local/lib/python3.6/dist-packages/third_party/nvml"
else:
    os.environ["PYTHONPATH"]+=":/home_duplicate/rarunachalam/miniconda3/envs/tf_py_36/lib/python3.6/site-packages/third_party/nvml" # FIX MINICONDA PATH

### 2.4 Reset env variables <a class="anchor" id="head-2-4"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
%env NUM_GPUS=1

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/ColabNotebooks
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise("Error, enter the path of the colab notebooks repo correctly")
%env EXPERIMENT_DIR=/content/drive/MyDrive/results/gesturenet
%env DATA_DIR=/content/drive/MyDrive/gesturenet_data/

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/gesturenet/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

### 3. Dataset convert <a class="anchor" id="head-3"></a>

* Update the `dataset_config.json` and `dataset_experiment_config.json` spec files
* Create the crop and json using the gesturenet dataset_convert 

*Note: Crops and dataset json only need to be generated once.*

In [ ]:
print("Hand crop generation spec file")
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/dataset_config.json
!cat $SPECS_DIR/dataset_config.json

In [ ]:
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/dataset_experiment_config.json
print("Dataset experiment spec file")
!cat $SPECS_DIR/dataset_experiment_config.json

In [ ]:
!tao gesturenet dataset_convert --dataset_spec $SPECS_DIR/dataset_config.json \
                                --k_folds 0 \
                                --experiment_spec $SPECS_DIR/dataset_experiment_config.json \
                                --output_filename $DATA_DIR/data.json \
                                --experiment_name v1

In [ ]:
# Check to see if proper json file is generated.
!if [ ! -f $DATA_DIR/data.json ]; then echo "Json file was not generated properly."; else echo "Json was generated properly."; fi

## 4. Provide training specification <a class="anchor" id="head-4"></a>

* Dataset configuration
    * In order to load the data properly, you will need to change the `dataset:data_path` to the generated `json` (folder and file) file generated in part B above. By default it is located at `$SPECS_DIR/data.json`
    * Update number of classes and class number to name map
* Pre-trained models. There is an optional parameter to load head of model. Only set `add_new_head: false` if you want to finetune on dataset with same gestures as pretrained model. Please ensure the gesture class to index map matches pretrained model.
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.|

In [ ]:
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/train_spec.json
!sed -i "s|EXPERIMENT_DIR_PATH|$EXPERIMENT_DIR/|g" $SPECS_DIR/train_spec.json
!cat $SPECS_DIR/train_spec.json

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the encryption key

In [ ]:
!tao gesturenet train -e $SPECS_DIR/train_spec.json \
                      -k $KEY

## 6. Evaluate the trained model <a class="anchor" id="head-6"></a>

* Please update model path to location trained model is saved at

In [ ]:
!tao gesturenet evaluate -e $EXPERIMENT_DIR/model/train_spec.json \
                         -m $EXPERIMENT_DIR/model/model.tlt \
                         -k $KEY

## 7. Inference <a class="anchor" id="head-7"></a>
In this section, we run the `gesturenet inference` tool to generate inferences on the trained models. Please ensure the spec file `inference.json` is configured correctly. 

In [ ]:
!tao gesturenet inference -e $EXPERIMENT_DIR/model/train_spec.json \
                          -m $EXPERIMENT_DIR/model/model.tlt \
                          -k $KEY \
                          --image_root_path $DATA_DIR \
                          --data_json $DATA_DIR/data.json \
                          --data_type kpi_set \
                          --results_dir $EXPERIMENT_DIR/model

In [ ]:
import os
import cv2
import IPython.display
import json
import PIL.Image

json_spec_path = os.path.join(os.environ.get('DATA_DIR'), 'data.json')
data_type = "kpi_set"
result_file = os.path.join(os.environ.get('EXPERIMENT_DIR'), 'model/results.txt')
model_spec_path = os.path.join(os.environ.get('EXPERIMENT_DIR'), 'model/train_spec.json')

# Read in json spec.
with open(json_spec_path, 'r') as file:
    full_spec = json.load(file)
spec = full_spec[data_type]

# Read in model spec.
with open(model_spec_path, 'r') as file:
    model_spec = json.load(file)

class_labels = model_spec['dataset']['classes']

results = open(result_file, 'r')

images = spec['images']

for image_dict in images:

    image_path = os.path.join(os.environ.get('DATA_DIR'), image_dict['full_image_path'])
    bbox = image_dict['bbox_coordinates']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Get corners of rectangle.
    upper_left = tuple(bbox[0])
    bottom_right = tuple(bbox[3])
    # draw rectangle onto image.
    cv2.rectangle(image, upper_left, bottom_right, (0, 255, 0), 2)

    if image is None:
        results.readline()
        continue
    image_result = results.readline()
    prediction = image_result.split(' ')[1]
    # Get class label.
    label = list(class_labels.keys())[list(class_labels.values()).index(int(prediction))]
    # Get bottom right corner.
    x = 0
    y = image.shape[0]-5
    # Display Image.
    image = cv2.putText(image, label, (x, y), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 24, 8))
    IPython.display.display(PIL.Image.fromarray(image))